<a href="https://colab.research.google.com/github/steghoja/Google2Csv/blob/master/Google2Csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##GoogleScraper

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def simpleGoogleSearch(query, start):
  results = []

  query = query.replace(' ', '+')
  URL = f"https://scholar.google.com/scholar?q={query}&start={start}&hl=en"

  # desktop user-agent
  USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.16; rv:84.0) Gecko/20100101 Firefox/84.0"
  ACCEPT = "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
  ACCEPT_ENCODING = "gzip, deflate, br"
  
  headers = {"user-agent" : USER_AGENT,
             "accept" : ACCEPT,
             "accept-encoding" : ACCEPT_ENCODING
            }
  resp = requests.get(URL, headers=headers)

  if resp.status_code == 200:
    soup = BeautifulSoup(resp.content, "html.parser")

    for g in soup.find_all('div', class_='gs_ri'):
      anchors = g.find_all('a')

      if anchors:
        link  = anchors[0]['href']
        title = g.find('h3').text
        description = g.find('div', class_='gs_rs').text
        authors = g.find('div', class_='gs_a').text
        item  = {"title": title, "authors": authors, "description": description, "link": link}
        results.append(item)

  return results

In [3]:
def googleToPandas(googleQuery):
  resultsCounter  = 0
  resultsList     = []

  while True:
    pageResults = simpleGoogleSearch(googleQuery, resultsCounter)
    
    if not pageResults: break
    else: 
      resultsList.extend(pageResults)
      resultsCounter = resultsCounter + 10

  return pd.DataFrame(resultsList)

In [4]:
googleSearchQuery = "traceability information model" #@param {type:"string"}

In [5]:
results = googleToPandas(googleSearchQuery)
results

""


In [6]:
results.to_csv('GoogleResults.csv', index=False)
#results.to_excel('GoogleResults.xlsx', index=False)
#results.to_json('GoogleResults.jsonl', orient='records', lines=True)